# Tensorflow and ProtocolBuffer

## Object Serialization in Depth

<br>
<div>
<h3><font color=#C7741B>Dboy Liao</font></h3>
<ul>
<li><font color=#DA8321>qmalliao@gmail.com </font></li>
<li><a href="https://github.com/dboyliao">https://github.com/dboyliao</a></li>
<li><a href="https://www.facebook.com/dboyliao">https://www.facebook.com/dboyliao</a></li>
</ul>
</div>

In this talk, I'll try to explain how `tensorflow` internally use `protobuf` as serialization tool and how serialized object works under the hook.

If you are not familiar with `protobuf`, I wrote a simplified version of neural network serialization with `protobuf` over [here](https://medium.com/@dboyliao/簡介-google-protocol-buffer-1dd5fa2e1e69).

Oh, It's written in chinese :P

## Typical Use Case

- Training
  - save session status with `tf.train.Saver.save`
  - restore session with `tf.train.Saver.restore` for upcoming experiments


- Deployment
  - serve the model in a loadable binary format
  - `tf.GraphDef` (is it a `tf.Graph`?)

Let's dive into the source code of `tensorflow`!

### What does `tf.train.Saver.save` do?

[source code](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/training/saver.py#L1462)

Two required arguements:

1. **sess** (`tf.Session`): a tensorflow Session object
2. **save_path** (`str`): the path where to write model checkpoint files, used to generate files below
  - **`(save_path)`-`(global_step)`.meta**
  - **`(save_path)`.data-`(step)`-of-`(total)`**
  - **`(save_path)`-`(global_step)`.index**

- **`.meta`** file --> `MetaGraphDef` protobuf file
- **`.data-????-of-????`** and **`.index`** files --> protobuff file for restoring variables/tensors
- In the `saver.save` call, `.data` and `.index` files are created with a `sess.run` call which will invoke an operation node, created by `saver`'s builder (See `Saver.__init__` for detail)
- After that, a checkpoint path will be returned.

![write_tensors](images/saver_write_tensors.png)

### MetaGraph

Basically **`save_path`** is used to detemine:
1. the path and prefix of saved files
2. updating `checkpoint` file

Once `tf.train.Saver` generate files for variables/tensors, it'll then export a **meta graph**

The source code is as following:

![saver-save](images/saver_save.png)


`meta_graph_filename` is now something like `(save_path)-(global_step).meta`


### `tf.train.Saver.export_meta_graph`

The source code is as following:

![saver-export-meta](images/saver_export_meta_graph.png)

It's just a wrapper method for calling `export_meta_graph` function, which is defined in the same source file as `Saver`.

<img alt="export-meta" src="images/export_meta_graph.png">

Basically, what `meta_graph.export_scoped_meta_graph` does is to construct an instance of `MetaGraphDef`, a class created by protobuf compiler, and save it to disk with standard `protobuf` API. I list a few `.proto` files below, check it out if you want to see the **definitions** for these protobuff objects.

- [meta_graph.proto](https://github.com/tensorflow/tensorflow/blob/r1.3/tensorflow/core/protobuf/meta_graph.proto)
- [saver.proto](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/protobuf/saver.proto)
- [graph.proto](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/framework/graph.proto)
- [node_def.proto](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/framework/node_def.proto)
- [tensor.proto](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/framework/tensor.proto)

There are many more of them, good luck!

So, the calling stacks to `export_meta_graph` will looks like this:
```
saver.export_meta_graph
  --> export_meta_graph
    --> meta_graph.export_scoped_meta_graph
        1. build graph_def from default graph or given graph
        2. create a meta_graph_def by the graph_def
        3. save the meta_graph_def to disk (xxx.meta)
```

Similary, the calling stacks to `tf.train.import_meta_graph` will reverse above process as follwoing:
```
import_meta_grph
  --> meta_graph.read_meta_graph_file to reconstruct MetaGraphDef object
    --> meta_graph.import_scoped_meta_graph to construct a new Graph object
      --> return a Saver object.
```

Why returning a `Saver`? 

What is it for?

By now, we only reconstruct the graph from a metagraph and the variables in the graph are not yet initialized.

So, the saver here is for initializing/restoring the value of all the tensors in the graph.

### What does tf.Saver.restore do?

Very simple, run the restore operation for a given session to restore the state of the session.

![saver-restore](images/saver_restore.png)

## Summary for Session Saving/Restoration

1. Saving to Disk:
  - `saver.save`
    - saving tensor values to `.data` and `.index` file
    - saving metagraph to `.meta`
2. Restoration from Disk:
  - restore metagraph, a graph consists of saver and graph
    - `tf.train.import_meta_graph`
  - restore session with restored saver
    - `saver.restore`

## You Graph, Freeze!

As you can see, saving a session will create three files at a time, `.data`, `.index` and `.meta`.


It's convenient for prototyping, experiments..etc, but, as for me, I prefer saving my graph in a more compact format when I just want to *use* the graph (like running the trained graph in a mobile device). 

Fortunately, `tensorflow` provides some util functions you can use to **freeze** your graph into a more compat format, with just *one* output file.

### tensorflow.python.framework.graph_util.convert_variables_to_constants

- As what its name suggests, `convert_variables_to_constant` will do two things:
  1. `extract_sub_graph`: extract sub graph from given `GraphDef` using BFS (Breath First Search).
  2. construct a new `GraphDef` object and copy the value of all variables in the input `GraphDef` in given session.

![freeze-convert-const](images/convert_const.png)

The next step is simple, just save returned `GraphDef` object (`SerializeToString`):

![save-graphdef](images/save_graph_def.png)

Then load the graph as following:

![load-graphdef](images/load_graph_def.png)

### `tf.import_graph_def`

The source code is meaty, let me just summarize the core steps here:

1. Use **`tf.Graph.create_op`** to create/copy `Operation`s defined in given `GraphDef` into new graph object. [create_op](https://www.tensorflow.org/api_docs/python/tf/Graph#create_op)
2. Link all nodes together according to attribute value, `inputs`, defined in `node_def.proto`. ([source code](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/framework/node_def.proto))
  - Add inputs with `tf.Graph._add_control_input` or `tf.Graph._add_input`
  - Setup shapes for non-function operation node.
![import-graph-def-core](images/import_graph_def_core.png)

#### Step 1

![import-graph-def-1](images/import_graph_def_1.png)

#### Step 2

<img alt="import-graph-def-2" src="images/import_graph_def_2.png" height="500" width='500'>

My demo is about showing you how to serve your graph the **hard** way

You can simply use util scripts provided by `tensorflow`, [freeze_graph.py](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/tools/freeze_graph.py).

By now, you should be able to understand what is going on in that script.

# Thanks for Your Attention

## Q&A